In [ ]:
from lxml import etree as ET
import requests
from requests.auth import HTTPBasicAuth
import os
import getpass

In [ ]:
user = "uncw/web_services_vivo"
password = getpass.getpass()

In [ ]:
def write_response(endpoint, filename, limiter='', beta=False):
    if beta:
        base_url = "https://betawebservices.digitalmeasures.com/login/service/v4"
    else:
        base_url = 'https://webservices.digitalmeasures.com'
    response = requests.get(
        f"{base_url}{endpoint}{limiter}",
        auth=HTTPBasicAuth(user, password)
    )
    with open(f"output/{filename}.xml", 'w', encoding='utf8') as f:
        f.write(response.text.strip())

def get_usernames():
    usersEtree = ET.parse('output/users.xml')
    users = usersEtree.findall('.//User')
    usernames = [i.attrib['username'] for i in users]
    return usernames

def get_userfiles(usernames):
    existing_files = os.listdir('output/users')
    failed = set()
    for username in usernames:
        if f"{username}.xml" in existing_files:
            continue
        endpoint = f"/login/service/v4/SchemaData/INDIVIDUAL-ACTIVITIES-University/USERNAME:{username}"
        filename = f"users/{username}"
        write_response(endpoint, filename)
        
usernames = get_usernames()
get_userfiles(usernames)

In [ ]:
# check for error xmls

# userfiles = sorted(os.listdir('output/users/'))
# count = 0

# for file in userfiles:
#     eTree = ET.parse(f"output/users/{file}")
#     if "error" in eTree.getroot().tag.lower():
#         print(os.path.splitext(file)[0])
#         count += 1
# #         with open(f"output/users/{file}") as f:
# #             print(f.read())
# print(f"errored Users count: {count}")

In [ ]:
# userfiles = sorted(os.listdir('output/users'))
# len_Message, len_Record = set(), set()

all_2nd_tags = set()
for file in userfiles:
    etree = ET.parse(f"output/users/{file}")
    len_Message.add(len(etree.findall('Message')))
    len_Record.add(len(etree.findall('{http://www.digitalmeasures.com/schema/data}Record')))
    for i in etree.getroot().iterchildren():
        all_2nd_tags.add(i.tag)
print(all_2nd_tags)
# print(len_Message, len_Record)

In [ ]:
# record_children = set()
# userfiles = sorted(os.listdir('output/users'))

# for file in userfiles:
#     etree = ET.parse(f"output/users/{file}")
#     record_tag = etree.find('{http://www.digitalmeasures.com/schema/data}Record')
#     if not record_tag:
#         continue
#     child_tags = record_tag.getchildren()
#     for i in child_tags:
#         record_children.add(i.tag)

# for i in record_children:
#     print(i)

In [ ]:
nsmap = {'xmlns': "http://www.digitalmeasures.com/schema/data",
         'dmd': "http://www.digitalmeasures.com/schema/data-metadata"
        }
all_presentations = set()

def grab_all_presentations(file):
    etree = ET.parse(f"output/users/{file}")
    #     print([i for i in etree.getroot().iterchildren()])
    for i in etree.xpath('./xmlns:Record/xmlns:PRESENT', namespaces=nsmap):
        if i.attrib['id'] in all_presentations:
            print(file, i.attrib['id'])
            continue
        all_presentations.add(i.attrib['id'])

userfiles = sorted(os.listdir('output/users'))
for file in userfiles:
    grab_all_presentations(file)

In [ ]:
len(all_presentations)